In [15]:
from noise_dataset import NoiseDataset # importando nosso dataset de ruido
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from math import floor # para dividir dataset

In [16]:
# criando nossa rede neural
class NN(nn.Module):
	def __init__(self):
		super(NN, self).__init__()
		self.operation_sequence = nn.Sequential(
			nn.Linear(5, 10),
			nn.ReLU(),
			nn.Linear(10, 2)
		)

	def forward(self, x):
		result = self.operation_sequence(x)
		return result

In [17]:
#verificando dispositivo a ser utilizado
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo {device}")

Usando o dispositivo cpu


In [18]:
# criando nosso calculador de acuracia
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad(): # usamos no_grad para nao alterar parametros da rede com cada chamada.
    for data, target in loader:
      data = data.to(device)
      target = target.to(device)
      scores = model(data)
      _, predictions = scores.max(1)
      num_correct += (predictions == target).sum()
      num_samples += predictions.size(0)
  
  model.train()
  return num_correct / num_samples


In [33]:
# definindo variaveis importantes
operator_size = 5
learning_rate = 0.01
batch_size = 12
num_epochs = 4

In [23]:
# carregando nosso dataset
dataset = NoiseDataset()
size = len(dataset)
train, test = torch.utils.data.random_split(dataset, lengths=[floor(0.7*size), floor(0.3*size)])
train_data = DataLoader(train, batch_size=batch_size, shuffle=True)
test_data = DataLoader(test, batch_size=batch_size, shuffle=True)

In [24]:
model = NN().to(device)

In [28]:
# criando loss function e otimizador
loss = nn.CrossEntropyLoss(weight=torch.Tensor([6.5, 1])) 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
# treinando nossa rede
for epoch in range(num_epochs):
  for i, (data, target) in enumerate(train_data):
    # definindo tensores para CUDA:
    data = data.to(device)
    target = target.to(device)
  
    preds = model(data)
    error = loss(preds, target)
		
    optimizer.zero_grad() # zerando gradiente para atualizar logo em seguida
    error.backward()
    optimizer.step()

In [35]:
check_accuracy(test_data, model)

tensor(0.9803)

In [36]:
def check_error(loader, model, loss):
  num_correct = 0
  num_samples = 0
  model.eval()
  running_error=0.0
  with torch.no_grad(): # usamos no_grad para nao alterar parametros da rede com cada chamada.
    for data, target in loader:
      data = data.to(device)
      target = target.to(device)
      scores = model(data)
      _, predictions = scores.max(1)
      num_correct += (predictions == target).sum()
      num_samples += predictions.size(0)
      running_error += loss(scores, target).item()
  model.train()
  return running_error/num_samples

In [37]:
check_error(test_data, model, loss)

0.00886502424708257

In [38]:
torch.save(model.state_dict(), 'model.st')